In [ ]:
pip install replicate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


In [ ]:
translation_system_prompt = {"ar2en" : "You are an expert Arabic to English translator. Translate the following text to English maintaining all key information. Only output the translation.",
                             "en2ar" : "You are an expert English to Arabic translator. You will be given a question in Arabic and its answer in English, translate the answer to Arabic maintaining all key information and in style addressing the question. Only output the answer translation."}

system_prompt = """
You are an intelligent research assistant that answers questions based on information from texts. Your primary goal is accuracy, and you should never fabricate information or make assumptions beyond the provided context.

Task:

1. Carefully read the question and understand the intent.
2. Thoroughly analyze the provided passages (text chunks).
3. If the answer is explicitly stated or logically inferred from the passages, provide an accurate and concise answer.
4. If there is insufficient information in the passages to answer the question definitively, respond with "I do not know".

Output the answer after "Answer:". Then any additional info after "Note:"
STRICTLY Never speculate, make assumptions, or generate information that is not directly supported by the given passages.
"""

In [ ]:
import pandas as pd
from sentence_transformers import util, SentenceTransformer
import replicate
import numpy as np
import ast
from tqdm import tqdm
import re
import torch

# Function to calculate cosine similarities and retrieve sentences
def retrieve_top_sentences(question_embedding, all_sentences_embeddings, gold_idx, k=20):
    similarities = util.cos_sim(question_embedding, all_sentences_embeddings)
    rank = (similarities[0] >= similarities[0, gold_idx]).sum()
    top_k_indices = similarities[0].argsort(descending=True)[:k]
    return df_documents.iloc[top_k_indices], rank.item()

def format_prompt(question, context):
  PROMPT = f"Question:\n{question}\nPassages:\n"
  for i in range(len(context)):
    PROMPT+= f"{i+1}- {context[i]}\n"
  return PROMPT

def get_answer(model, prompt):
    attempts = 0
    max_attempts = 3

    while attempts < max_attempts:
        try:
            response = replicate.run(
                model,
                input={
                    "prompt": prompt,
                    "system_prompt": system_prompt,
                },
            )
            answer = "".join(response)
            pattern = r"Answer:(.*?)Note:"
            match = re.search(pattern, answer, re.S)
            return match.group(1).strip()  # Assuming response is a list of strings that needs to be joined
        except Exception as e:
            print(f"Attempt {attempts + 1} failed: {e}")
            attempts += 1
            if attempts == max_attempts:
                return "NA"

def get_translation(text, lang):
    attempts = 0
    max_attempts = 3
    while attempts < max_attempts:
        try:
            response = replicate.run(
                "meta/meta-llama-3-8b-instruct",
                input={
                    "prompt": text,
                    "system_prompt": translation_system_prompt[lang],
                },
            )
            return "".join(response)
        except Exception as e:
            print(f"Attempt {attempts + 1} failed: {e}")
            attempts += 1
            if attempts == max_attempts:
                return "NA"

In [ ]:
df_questions = pd.read_csv("dataset_with_embeddings.csv")
df_documents = pd.read_csv("document_dataset.csv")

In [ ]:
for idx, row in tqdm(df_documents.iterrows(), total=df_documents.shape[0], desc="Processing Sentences"):
  sentence = row["sentence"]
  answer = get_translation(sentence, "ar2en")
  df_documents.loc[idx, "english_sentence"] = answer

Processing Sentences: 100%|██████████| 514/514 [14:18<00:00,  1.67s/it]


In [ ]:
for idx, row in tqdm(df_questions.iterrows(), total=df_questions.shape[0], desc="Processing Sentences"):
  question = row["question_formal"]
  answer = get_translation(question, "ar2en")
  df_questions.loc[idx, "english_question_formal"] = answer

Processing Sentences: 100%|██████████| 150/150 [01:54<00:00,  1.31it/s]


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer('intfloat/multilingual-e5-large', device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

In [ ]:
df_documents = pd.read_csv("en_document_dataset.csv")

In [ ]:
embeddings = model.encode(df_documents["english_sentence"].tolist())
df_documents["english_embedding"] = embeddings.tolist()

In [ ]:
embeddings = model.encode(df_questions["english_question_formal"].tolist())
df_questions["english_question_formal_embedding"] = embeddings.tolist()

In [ ]:
# df_questions.to_csv("en_dataset_with_embeddings.csv", index=False)
df_documents.to_csv("en_document_dataset.csv", index=False)

In [ ]:
df_questions = pd.read_csv("en_dataset_with_embeddings.csv")
df_documents = pd.read_csv("en_document_dataset.csv")

def str_to_array(s):
    # Using ast.literal_eval to safely evaluate the string
    return np.array(ast.literal_eval(s))

df_questions['english_question_formal_embedding'] = df_questions['english_question_formal_embedding'].apply(str_to_array)
# df_questions['question_conversational_embedding'] = df_questions['question_conversational_embedding'].apply(str_to_array)
df_documents['english_embedding'] = df_documents['english_embedding'].apply(str_to_array)

# Iterate over questions
for idx, row in tqdm(df_questions.iterrows(), total=df_questions.shape[0], desc="Processing Questions"):
    # for question_type in ["formal", "conversational"]:
    for question_type in ["formal"]:
        ar_question = row[f"question_{question_type}"]
        question = row[f"english_question_{question_type}"]
        embedding = row[f"english_question_{question_type}_embedding"]
        if type(row['chunck']) == str:
          if "," in row['chunck']:
            true_idx = int(row['chunck'][:2])
          else:
            true_idx = int(row['chunck'])
        else:
          true_idx = 0
        # Retrieve top 20 sentences
        top_20, rank = retrieve_top_sentences(embedding, df_documents['english_embedding'].tolist(), true_idx, 20)
        context_r1 = format_prompt(question, top_20['sentence'].iloc[:10].tolist())
        # context_r2 = format_prompt(question, top_20['sentence'].iloc[10:20].tolist())

        # Calculate rank and retrieval status
        df_questions.loc[idx, f"{question_type}_rank"] = rank
        df_questions.loc[idx, f"{question_type}_retrieved"] = 1 if rank <= 10 else 0.5 if rank <= 20 else 0
        # Get answers using both model versions
        model = f"meta/meta-llama-3-70b-instruct"
        answer_r1 = get_answer(model, context_r1)
        # answer_r2 = "None" if "لا ادري" in answer_r1 else get_answer(model, context_r2)
        df_questions.loc[idx, f"english_{question_type}_answer_r1"] = answer_r1
        # Log answers and labels
        answer = get_translation(f"Question (Arabic):\n{ar_question}\nAnswer (English):\n{answer_r1}", "en2ar")
        df_questions.loc[idx, f"{question_type}_answer_r1"] = answer
        # df_questions.loc[idx, f"{model_size}_{question_type}_answer_r2"] = answer_r2

Processing Questions:   0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/sentence_transformers/util.py:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:274.)
  a = torch.tensor(a)
Processing Questions:  13%|█▎        | 20/150 [01:21<07:56,  3.67s/it]

Attempt 1 failed: 


Processing Questions:  25%|██▍       | 37/150 [03:33<08:09,  4.33s/it]

Attempt 1 failed: 


Processing Questions:  35%|███▍      | 52/150 [05:40<07:15,  4.44s/it]

...

Processing Questions:  67%|██████▋   | 100/150 [42:54<44:34, 53.48s/it]

Attempt 1 failed: 


Processing Questions:  77%|███████▋  | 116/150 [55:17<20:47, 36.70s/it]

Attempt 1 failed: 


Processing Questions: 100%|██████████| 150/150 [1:15:41<00:00, 30.28s/it]


In [ ]:
df_questions.to_excel("translation_results.xlsx", index=False)

In [ ]:
df_questions.columns

Index(['question_formal', 'question_conversational', 'answer', 'context',
       'page', 'chunck', 'question_formal_embedding',
       'question_conversational_embedding', 'english_question_formal',
       'english_question_formal_embedding', 'formal_rank', 'formal_retrieved',
       'english_formal_answer_r1', 'formal_answer_r1'],
      dtype='object')

In [ ]:
column_order = ['question_formal', 'english_question_formal', 'question_conversational', 'context', 'answer',  'formal_retrieved', 'formal_answer_r1', 'english_formal_answer_r1',
       'page', 'chunck', 'question_formal_embedding',
       'question_conversational_embedding',
       'english_question_formal_embedding', 'formal_rank']
df_questions = df_questions[column_order]